In [20]:
import pandas as pd
import numpy as np

from requests import get
from bs4 import BeautifulSoup
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import re
import time

from env import github_token, github_username

In [21]:
df = pd.read_csv('large_scraped_data.csv')

In [22]:
df

,Unnamed: 0,repo,language,readme_contents
0,0,MinecraftForge/MinecraftForge,Java,NaN
1,1,fogleman/Minecraft,Python,# Minecraft\n\nSimple Minecraft-inspired demo ...
2,2,itzg/docker-minecraft-server,Shell,[![Docker Pulls](https://img.shields.io/docker...
3,3,overviewer/Minecraft-Overviewer,Python,====================================\nMinecraf...
4,4,TheGreyGhost/MinecraftByExample,Java,MinecraftByExample [1.16.4]\n=================...
...,...,...,...,...
995,995,GeyserMC/PacketLib,Java,# PacketLib\nPacketLib is a library for packet...
996,996,Nic4Las/Minecraft-Enderite-Mod,Java,# Enderite Mod\n\n[![](http://cf.way2muchnoise...
997,997,grahamedgecombe/minecraft-installer,Groff,NaN
998,998,espertus/blockly-lua,JavaScript,Blockly Lua allows you to program ComputerCraf...


In [24]:
df.language.value_counts()


Java                 427
JavaScript            95
Python                78
C#                    46
C++                   37
Shell                 36
PHP                   34
Kotlin                23
TypeScript            22
Go                    19
C                     16
Rust                  15
HTML                  10
ZenScript              6
GLSL                   6
Lua                    6
Dockerfile             5
Ruby                   5
CSS                    5
Scala                  5
Jupyter Notebook       4
Batchfile              3
HCL                    3
CoffeeScript           3
Visual Basic .NET      2
Elixir                 2
mcfunction             2
TeX                    2
Mustache               2
PowerShell             2
CMake                  1
R                      1
Perl                   1
Haskell                1
GDScript               1
SCSS                   1
AutoHotkey             1
PureBasic              1
Vue                    1
Swift                  1
